In [ ]:
using DelimitedFiles
using Plots
using LaTeXStrings
using Printf

In [ ]:
using Dates

In [ ]:
function time2str(t)
    ms = Millisecond(round(1000 * t))
    dt = convert(DateTime, ms)
    tt = convert(Time, dt)
    return string(tt)
end

---
# CSV

In [ ]:
using CSV

In [ ]:
df = CSV.read("results.txt", delim=" ", ignorerepeated=true)

In [ ]:
df[df.method.=="IAPG",[:n, :γ, :k, :fgs, :time]]

---
# DataFrames

In [ ]:
using DataFrames

In [ ]:
time2str(sum(df.time))

In [ ]:
describe(df)

In [ ]:
dfgroupbymethod = groupby(df, :method);

In [ ]:
for i in eachindex(dfgroupbymethod)
    display(i.method)
    display(unstack(dfgroupbymethod[i], :n, :γ, :fgs))
end

---
# NamedArrays

In [ ]:
using NamedArrays

In [ ]:
ns = collect(100:100:800)
γs = collect(0.1:0.1:1.0)

for i in eachindex(dfgroupbymethod)
    println(i.method)
    A = Array{Int}(unstack(dfgroupbymethod[i], :n, :γ, :fgs))
    N = NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))
    println(N)
    println()
end

---
# heatmap

In [ ]:
function myunstack(dfgrouped, method, category)
    ns = collect(100:100:800)
    γs = collect(0.1:0.1:1.0)
    df = dfgroupbymethod[(method=method,)]
    T = eltype(getproperty(df, category))
    A = Array{T}(unstack(df, :n, :γ, category))
    return NamedArray(A[:,2:end], (ns, γs), ("n", "γ"))
end

In [ ]:
IAPGfgcalls = myunstack(dfgroupbymethod, "IAPG", :fgs)

In [ ]:
IRfgcalls = myunstack(dfgroupbymethod, "IR", :fgs)

In [ ]:
IERfgcalls = myunstack(dfgroupbymethod, "IER", :fgs)

In [ ]:
X = IAPGfgcalls - IERfgcalls

In [ ]:
extrema(X)

In [ ]:
ns = [string(n) for n=100:100:800]
γs = [@sprintf("%.1f", γ) for γ=0.1:0.1:1.0]
plt = heatmap(ns, γs, IAPGfgcalls - IERfgcalls, 
    size=(800, 600),
    #aspect_ratio=1.0,
    xlabel=L"n",
    ylabel=L"\gamma",
    title="Reduction in function evaluations (IER v. IAPG)",
    c=:RdBu,
    clims=(-12000, 12000),
)

In [ ]:
savefig(plt, "figs/heatmap.png")

In [ ]:
clibraries()

In [ ]:
showlibrary(:colorbrewer)

In [ ]:
for colorlib in clibraries()
    println(colorlib)
    display(showlibrary(colorlib))
end

---
# BenchmarkProfiles

In [ ]:
using BenchmarkProfiles

In [ ]:
category = :fgs
IAPG = myunstack(dfgroupbymethod, "IAPG", category)
IR = myunstack(dfgroupbymethod, "IR", category)
IER = myunstack(dfgroupbymethod, "IER", category)

In [ ]:
T = [IAPG.array[:] IER.array[:] IR.array[:]]
solvernames = ["IAPG", "IER", "IR"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)
savefig(plt, "figs/performance_profile.pdf")
plt

In [ ]:
T = [IAPG.array[:] IER.array[:]]
solvernames = ["IAPG", "IER"]

plt = performance_profile(T, solvernames, logscale=false, 
    size=(800, 600),
    legend=:bottomright, 
    style=:auto,
    linecolor=:black)
savefig(plt, "figs/performance_profile2.pdf")
plt